In [ ]:
#default_exp en_task.run_folds_task1

In [ ]:
#export
import os

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import transformers

import Hasoc.config as config
import Hasoc.utils.utils as utils
import Hasoc.utils.engine as engine
import Hasoc.model.model as model
import Hasoc.dataset.dataset as dataset

from functools import partial
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
#export
SEED = 42
utils.seed_everything(SEED)

In [ ]:
#export
df = pd.read_csv(config.DATA_PATH/'fold_df.csv')

In [ ]:
#hide
df.head(2)

,tweet_id,text,task1,task2,ID,kfold_task1,kfold_task2
0,1.126953e+18,"We need a word for ‘going somewhere alone,sitt...",NOT,NONE,hasoc_2020_en_1503,2,0
1,1.123482e+18,RT @RiverCityLabs: Come and work from our spac...,NOT,NONE,hasoc_2020_en_3570,2,0


In [ ]:
#hide
df.shape

(3708, 7)

In [ ]:
#export
le = LabelEncoder()
le.fit_transform(df.task1)
le.classes_

array(['HOF', 'NOT'], dtype=object)

In [ ]:
#export
df['task1_encoded'] = le.transform(df.task1.values)

In [ ]:
os.listdir(config.DATA_PATH)

['en_task_a', 'raw', 'fold_df.csv', '.ipynb_checkpoints']

In [ ]:
#export
test_df = pd.read_csv(config.DATA_PATH/'en_task_a/english_test.csv')

In [ ]:
#hide
test_df.head()

,tweet_id,text,task1,task2,ID
0,1130081762154090497,RT @delmiyaa: Samini resetting the show and mo...,NOT,NONE,hasoc_2020_en_2713
1,1130048316807491584,@Swxnsea how do you know that he’s left?,HOF,NONE,hasoc_2020_en_3874
2,1123657766143504386,Tried to get Divock Origi on a free seeing as ...,NOT,NONE,hasoc_2020_en_281
3,1126782963042013186,RT @nutclusteruwu: that....is yalls stupid whi...,HOF,PRFN,hasoc_2020_en_2026
4,1130159113529434113,&amp; IT DID. But a bitch got big girls things...,HOF,PRFN,hasoc_2020_en_4023


In [ ]:
#export
def run(fold, num_epochs=6):
    #export
    NUM_EPOCHS = num_epochs
    train_df = df.query(f'kfold_task1!={fold}').reset_index(drop=True)
    valid_df = df.query(f'kfold_task1=={fold}').reset_index(drop=True)

    #export
    train_dl = utils.create_loader(train_df.text.values, train_df.task1_encoded, bs=config.TRAIN_BATCH_SIZE)
    valid_dl = utils.create_loader(valid_df.text.values, valid_df.task1_encoded, bs=config.VALID_BATCH_SIZE)

    #export
    modeller = model.HasocModel(len(le.classes_))

    #export
    model_params = list(modeller.named_parameters())

    #export
    # we don't want weight decay for these
    no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']

    optimizer_params = [
        {'params': [p for n, p in model_params if n not in no_decay],
        'weight_decay':0.001},
        #  no weight decay should be applied
        {'params': [p for n, p in model_params if n in no_decay],
        'weight_decay':0.0}
    ]

    #export
    lr = config.LR

    #export
    optimizer = AdamW(optimizer_params, lr=lr)

    #export
    num_train_steps = int(len(df) / config.TRAIN_BATCH_SIZE * config.NUM_EPOCHS)

    #export
    scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                                    num_warmup_steps=30,
                                                    num_training_steps=num_train_steps-30)

    #export
    fit = engine.BertFitter(modeller, (train_dl, valid_dl), optimizer, nn.CrossEntropyLoss(), partial(f1_score, average='macro'), config.DEVICE, scheduler=scheduler, log_file='en_task1_log.txt')
    # fit = engine.BertFitter(modeller, (train_dl, valid_dl), optimizer, utils.LabelSmoothingCrossEntropy(), partial(f1_score, average='macro'), config.DEVICE, scheduler=scheduler, log_file='en_task1_log.txt')

    fit.fit(NUM_EPOCHS, model_path=os.path.join(config.MODEL_PATH/f'en_task1_{fold}.pth'), show_graph=False)

    #export
    test_dl = utils.create_loader(test_df.text.values, lbls=[None]*len(test_df.text.values), bs=config.VALID_BATCH_SIZE, is_test=True)

    #export
    modeller = model.HasocModel(len(le.classes_))
    modeller.load_state_dict(torch.load(config.MODEL_PATH/f'en_task1_{fold}.pth'))

    #export
    preds = engine.get_preds(test_dl.dataset, test_dl, modeller, config.DEVICE, ensemble_proba=True)

    np.save(os.path.join('..', 'outputs', f'submission_EN_A_{fold}.npy'), preds)

In [ ]:
#export
for i in range(5):
    run(i)

epoch,train_loss,valid_loss,metric,time
1,0.497978,0.286790,0.877580,0:2:38
2,0.253114,0.284326,0.921859,0:3:10
3,0.194607,0.287788,0.916350,0:3:3
4,0.145255,0.304420,0.911365,0:3:7
5,0.118769,0.298425,0.916063,0:3:2
6,0.108368,0.298297,0.916163,0:3:6


epoch,train_loss,valid_loss,metric,time
1,0.529932,0.298403,0.885167,0:3:10
2,0.241912,0.314131,0.907011,0:2:0
3,0.183218,0.337859,0.897978,0:1:40
4,0.137593,0.352730,0.899392,0:1:40
5,0.113909,0.355312,0.899159,0:1:40
6,0.103607,0.355367,0.899154,0:2:1


epoch,train_loss,valid_loss,metric,time
1,0.519209,0.271797,0.899546,0:3:10
2,0.246114,0.273747,0.921805,0:3:7
3,0.193305,0.285850,0.913146,0:3:5
4,0.149939,0.286022,0.915597,0:3:7
5,0.125278,0.293036,0.914432,0:3:4
6,0.116158,0.293185,0.914407,0:3:6


epoch,train_loss,valid_loss,metric,time
1,0.473475,0.287212,0.885153,0:2:0
2,0.257233,0.311727,0.898650,0:3:10
3,0.197072,0.337481,0.891685,0:3:3
4,0.157409,0.303837,0.896370,0:3:6
5,0.129853,0.304509,0.901938,0:3:6
6,0.123097,0.304524,0.902057,0:3:10


epoch,train_loss,valid_loss,metric,time
1,0.482765,0.281277,0.876744,0:3:11
2,0.242957,0.295409,0.894875,0:2:37
3,0.186691,0.316428,0.897054,0:1:44
4,0.149008,0.299309,0.898379,0:1:44
5,0.123504,0.292906,0.904369,0:1:44
6,0.116357,0.292770,0.904497,0:1:44


In [ ]:
#export
def ensemble():
    preds_0 = np.load(os.path.join('..', 'outputs', f'submission_EN_A_0.npy'))
    preds_1 = np.load(os.path.join('..', 'outputs', f'submission_EN_A_1.npy'))
    preds_2 = np.load(os.path.join('..', 'outputs', f'submission_EN_A_2.npy'))
    preds_3 = np.load(os.path.join('..', 'outputs', f'submission_EN_A_3.npy'))
    preds_4 = np.load(os.path.join('..', 'outputs', f'submission_EN_A_4.npy'))

    preds = (preds_0 + preds_1 + preds_2 + preds_3 + preds_4) / 5

    preds = le.inverse_transform(torch.tensor(preds).argmax(dim=-1).numpy())

    #export
    sub = pd.read_csv(config.DATA_PATH/'en_task_a/english_test.csv')

    #export
    submission_en_task1_df = test_df.drop(columns=['text', 'task1', 'task2']).copy()

    #export
    submission_en_task1_df['task1'] = preds

    #export
    submission_en_task1_df.to_csv(os.path.join('..', 'outputs', f'submission_EN_A.csv'), index=False)

In [ ]:
#export
ensemble()

In [ ]:
pd.read_csv(os.path.join('..', 'outputs', f'submission_EN_A.csv'))

,tweet_id,ID,task1
0,1130081762154090497,hasoc_2020_en_2713,NOT
1,1130048316807491584,hasoc_2020_en_3874,NOT
2,1123657766143504386,hasoc_2020_en_281,NOT
3,1126782963042013186,hasoc_2020_en_2026,HOF
4,1130159113529434113,hasoc_2020_en_4023,HOF
...,...,...,...
809,1127061607433900032,hasoc_2020_en_1212,NOT
810,1123685826074951681,hasoc_2020_en_3435,HOF
811,1126882552587927552,hasoc_2020_en_3987,NOT
812,1130294488859996160,hasoc_2020_en_1176,NOT
